# Import necessary packages

In [1]:
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.metrics import mean_absolute_error, mean_squared_error

from timeit import default_timer as timer

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Import original data and preprocessed data

In [3]:
sales_train_val = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/sales_train_validation.csv')

In [4]:
sales_train_val = sales_train_val[(sales_train_val['state_id'] == 'CA') & (sales_train_val['cat_id'] == 'HOUSEHOLD')]

In [5]:
INPUT_DIR_2 = '/content/drive/MyDrive/Colab Notebooks/List of Product ID according to 4 demand patterns/California/Household/2-year'
list_intermittent = pd.read_csv(f'{INPUT_DIR_2}/Intermittent_ID_2_Year_Data.csv')
list_lumpy = pd.read_csv(f'{INPUT_DIR_2}/Lumpy_ID_2_Year_Data.csv')
list_erratic = pd.read_csv(f'{INPUT_DIR_2}/Erratic_ID_2_Year_Data.csv')
list_smooth = pd.read_csv(f'{INPUT_DIR_2}/Smooth_ID_2_Year_Data.csv')

list_intermittent = list_intermittent['0'].values.tolist()
list_lumpy = list_lumpy['0'].values.tolist()
list_erratic = list_erratic['0'].values.tolist()
list_smooth = list_smooth['0'].values.tolist()

In [6]:
sales_intermittent = sales_train_val[sales_train_val.id.isin(list_intermittent)]
sales_lumpy = sales_train_val[sales_train_val.id.isin(list_lumpy)]
sales_erratic = sales_train_val[sales_train_val.id.isin(list_erratic)]
sales_smooth = sales_train_val[sales_train_val.id.isin(list_smooth)]

# User-defined functions to calculate Metrics and Croston algorithm

In [7]:
ROUNDING_DECIMAL = 4

def mase_calculation(ts, prediction):
    divisor = 0
    for i in range(1, ts.shape[0]):
        divisor = divisor + abs(ts.iloc[i] - ts.iloc[i-1])
    divisor = divisor/(ts.shape[0] - 1)
    diff    = abs(ts - prediction[:ts.shape[0]])/divisor
    mase    = diff.mean()
    return mase

def mape_calculation(actual, pred): 
    if not all([isinstance(actual, np.ndarray), isinstance(pred, np.ndarray)]):
        actual, pred = np.array(actual), np.array(pred)
    mask = (actual != 0)
    return round((np.fabs(actual - pred)/actual)[mask].mean()*100, ROUNDING_DECIMAL)

def wmape_calculation(actual, pred):
    if not all([isinstance(actual, np.ndarray), isinstance(pred, np.ndarray)]):
        actual, pred = np.array(actual), np.array(pred)
    return round((np.sum(np.absolute(actual-pred))/np.sum(actual))*100, ROUNDING_DECIMAL)

def smape_calculation(actual, predicted):
    if not all([isinstance(actual, np.ndarray), isinstance(predicted, np.ndarray)]):
        actual, predicted = np.array(actual), np.array(predicted)
    return round(np.mean(np.abs(predicted - actual) / ((np.abs(predicted) + np.abs(actual))/2))*100, ROUNDING_DECIMAL)

In [8]:
def Croston(ts,extra_periods=1,alpha=0.4):
    d = np.array(ts) # Transform the input into a numpy array
    cols = len(d) # Historical period length
    d = np.append(d,[np.nan]*extra_periods) # Append np.nan into the demand array to cover future periods
    
    #level (a), periodicity(p) and forecast (f)
    a,p,f = np.full((3,cols+extra_periods),np.nan)
    q = 1 #periods since last demand observation
    
    # Initialization
    first_occurence = np.argmax(d[:cols]>0)
    a[0] = d[first_occurence]
    p[0] = 1 + first_occurence
    f[0] = a[0]/p[0]
    # Create all the t+1 forecasts
    for t in range(0,cols):        
        if d[t] > 0:
            a[t+1] = alpha*d[t] + (1-alpha)*a[t] 
            p[t+1] = alpha*q + (1-alpha)*p[t]
            f[t+1] = a[t+1]/p[t+1]
            q = 1           
        else:
            a[t+1] = a[t]
            p[t+1] = p[t]
            f[t+1] = f[t]
            q += 1
       
    # Future Forecast 
    a[cols+1:cols+extra_periods] = a[cols]
    p[cols+1:cols+extra_periods] = p[cols]
    f[cols+1:cols+extra_periods] = f[cols]
                      
    df = pd.DataFrame.from_dict({"Demand":d,"Forecast":f,"Period":p,"Level":a,"Error":d-f})
    return df

# Set value for parameters

In [9]:
list_params = [round(item, 1) for item in list(np.arange(0.1, 1, 0.1))]

In [10]:
list_params

[0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]

In [11]:
start_train_date = '2014-04-11'
end_train_date = '2016-04-10'
n_pred_days = 14
num_train_needed = 50
validation_training_ratio = 0.95

# Pipeline for Croston

In [12]:
def Croston_output_all_params(sales_pattern, list_params, start_train_date, end_train_date, 
                       n_pred_days, num_train_needed, validation_training_ratio):
    
    sales_pattern_py = sales_pattern.copy()
    list_pattern_py = sales_pattern_py.id.unique().tolist()
    
    sales_pattern_py = sales_pattern_py.drop(['item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'], axis=1)
    df_pattern = sales_pattern_py.melt(['id'], var_name='Date').pivot(index = ['Date'], columns = 'id', values = 'value').reset_index()
    df_pattern.drop('Date', axis=1, inplace=True)
    df_pattern.index = pd.date_range('2011-01-29', periods=df_pattern.shape[0], freq="D")
    df_pattern.index.names = ['Date']
    df_pattern = df_pattern.astype('float64')
    
    train_data = df_pattern[(df_pattern.index >= start_train_date) & (df_pattern.index <= end_train_date)]
    test_data = df_pattern.iloc[n_pred_days*-1:]
    train_data_first_part = train_data.iloc[:int(len(train_data)*validation_training_ratio), :]
    train_data_second_part = train_data.iloc[int(len(train_data)*validation_training_ratio):, :]
    
    df_result_all_params = pd.DataFrame()
    for product in list_pattern_py:
        train_first = train_data_first_part[product]
        train_second = train_data_second_part[product]
        for i in list_params:
            predictions = list()
            history = [x for x in train_first[num_train_needed*-1:]]
            for t in range(len(train_second)):
                yhat = Croston(history, extra_periods=1, alpha=i)['Forecast'].iloc[-1]
                predictions.append(yhat)
                history.append(train_second[t])
            df_result_temp = pd.DataFrame({'Product': [product for count in range(len(train_second))],
                                            'Actual Data': train_second,
                                            'Forecast': predictions,
                                            'Alpha': [i for count in range(len(train_second))]})
            df_result_all_params = df_result_all_params.append(df_result_temp, ignore_index=True)
            
    return df_result_all_params

In [13]:
start = timer()

intermittent_df_result_all_params = Croston_output_all_params(sales_intermittent,
                                                      list_params,
                                                      start_train_date,
                                                      end_train_date, 
                                                      n_pred_days,
                                                      num_train_needed,
                                                      validation_training_ratio)

end = timer()

print('This line of code took {} minutes'.format((end-start) / 60))

This line of code took 11.546245583066668 minutes


In [14]:
intermittent_df_result_all_params

,Product,Actual Data,Forecast,Alpha
0,HOUSEHOLD_1_001_CA_1_validation,0.0,0.272723,0.1
1,HOUSEHOLD_1_001_CA_1_validation,0.0,0.272723,0.1
2,HOUSEHOLD_1_001_CA_1_validation,0.0,0.272723,0.1
3,HOUSEHOLD_1_001_CA_1_validation,0.0,0.272723,0.1
4,HOUSEHOLD_1_001_CA_1_validation,1.0,0.272723,0.1
...,...,...,...,...
1162165,HOUSEHOLD_2_516_CA_4_validation,0.0,0.072428,0.9
1162166,HOUSEHOLD_2_516_CA_4_validation,0.0,0.072428,0.9
1162167,HOUSEHOLD_2_516_CA_4_validation,0.0,0.072428,0.9
1162168,HOUSEHOLD_2_516_CA_4_validation,0.0,0.072428,0.9


In [15]:
intermittent_df_result_all_params.to_csv('Croston_Intermittent_All_Parameters_Hyperparameter_Tuning_Data.csv')

# Summay all metrics

In [16]:
def get_metrics_result_all_params(data):
    metrics_df = {}
    metrics_df['MASE'] = mase_calculation(data['Actual Data'], data['Forecast'])
    metrics_df['WMAPE'] = wmape_calculation(data['Actual Data'], data['Forecast'])
    metrics_df['SMAPE'] = smape_calculation(data['Actual Data'], data['Forecast'])
    metrics_df['MAPE'] = mape_calculation(data['Actual Data'], data['Forecast'])
    metrics_df['MAE'] = mean_absolute_error(data['Actual Data'], data['Forecast'])
    metrics_df['RMSE'] = np.sqrt(mean_squared_error(data['Actual Data'], data['Forecast']))
    return pd.Series(metrics_df)

In [17]:
start = timer()

df_result_metrics_all_params = intermittent_df_result_all_params.groupby(['Product', 'Alpha']).apply(get_metrics_result_all_params).reset_index()

end = timer()
print('This line of code took {} minutes'.format((end-start) / 60))

This line of code took 0.9345367814333334 minutes


In [18]:
df_result_metrics_all_params

,Product,Alpha,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE
0,HOUSEHOLD_1_001_CA_1_validation,0.1,1.223481,251.4932,191.9765,77.1624,0.271885,0.324024
1,HOUSEHOLD_1_001_CA_1_validation,0.2,1.111556,228.4865,192.8527,80.0938,0.247012,0.315403
2,HOUSEHOLD_1_001_CA_1_validation,0.3,1.065246,218.9673,193.2088,81.1990,0.236721,0.313042
3,HOUSEHOLD_1_001_CA_1_validation,0.4,1.042854,214.3644,193.3655,81.6165,0.231745,0.312586
4,HOUSEHOLD_1_001_CA_1_validation,0.5,1.032587,212.2541,193.4331,81.7239,0.229464,0.313106
...,...,...,...,...,...,...,...,...
31405,HOUSEHOLD_2_516_CA_4_validation,0.5,2.098249,431.3067,198.8560,88.1657,0.116569,0.173007
31406,HOUSEHOLD_2_516_CA_4_validation,0.6,1.979582,406.9141,198.9306,89.0227,0.109977,0.170523
31407,HOUSEHOLD_2_516_CA_4_validation,0.7,1.880075,386.4598,199.0080,89.8969,0.104449,0.168817
31408,HOUSEHOLD_2_516_CA_4_validation,0.8,1.798788,369.7508,199.0913,90.8228,0.099933,0.167830


# Check what products has unexpected metrics result

In [19]:
df_result_metrics_all_params[df_result_metrics_all_params['MASE'].isin([np.inf, -np.inf, np.nan])]

,Product,Alpha,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE
9,HOUSEHOLD_1_001_CA_2_validation,0.1,inf,inf,200.0,NaN,0.069348,0.069348
10,HOUSEHOLD_1_001_CA_2_validation,0.2,inf,inf,200.0,NaN,0.068120,0.068120
11,HOUSEHOLD_1_001_CA_2_validation,0.3,inf,inf,200.0,NaN,0.067659,0.067659
12,HOUSEHOLD_1_001_CA_2_validation,0.4,inf,inf,200.0,NaN,0.067935,0.067935
13,HOUSEHOLD_1_001_CA_2_validation,0.5,inf,inf,200.0,NaN,0.068966,0.068966
...,...,...,...,...,...,...,...,...
31369,HOUSEHOLD_2_515_CA_4_validation,0.5,NaN,NaN,NaN,NaN,0.000000,0.000000
31370,HOUSEHOLD_2_515_CA_4_validation,0.6,NaN,NaN,NaN,NaN,0.000000,0.000000
31371,HOUSEHOLD_2_515_CA_4_validation,0.7,NaN,NaN,NaN,NaN,0.000000,0.000000
31372,HOUSEHOLD_2_515_CA_4_validation,0.8,NaN,NaN,NaN,NaN,0.000000,0.000000


In [20]:
df_result_metrics_all_params[df_result_metrics_all_params['WMAPE'].isin([np.inf, -np.inf, np.nan])]

,Product,Alpha,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE
9,HOUSEHOLD_1_001_CA_2_validation,0.1,inf,inf,200.0,NaN,0.069348,0.069348
10,HOUSEHOLD_1_001_CA_2_validation,0.2,inf,inf,200.0,NaN,0.068120,0.068120
11,HOUSEHOLD_1_001_CA_2_validation,0.3,inf,inf,200.0,NaN,0.067659,0.067659
12,HOUSEHOLD_1_001_CA_2_validation,0.4,inf,inf,200.0,NaN,0.067935,0.067935
13,HOUSEHOLD_1_001_CA_2_validation,0.5,inf,inf,200.0,NaN,0.068966,0.068966
...,...,...,...,...,...,...,...,...
31369,HOUSEHOLD_2_515_CA_4_validation,0.5,NaN,NaN,NaN,NaN,0.000000,0.000000
31370,HOUSEHOLD_2_515_CA_4_validation,0.6,NaN,NaN,NaN,NaN,0.000000,0.000000
31371,HOUSEHOLD_2_515_CA_4_validation,0.7,NaN,NaN,NaN,NaN,0.000000,0.000000
31372,HOUSEHOLD_2_515_CA_4_validation,0.8,NaN,NaN,NaN,NaN,0.000000,0.000000


In [21]:
df_result_metrics_all_params[df_result_metrics_all_params['SMAPE'].isin([np.inf, -np.inf, np.nan])]

,Product,Alpha,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE
234,HOUSEHOLD_1_009_CA_1_validation,0.1,NaN,NaN,NaN,NaN,0.0,0.0
235,HOUSEHOLD_1_009_CA_1_validation,0.2,NaN,NaN,NaN,NaN,0.0,0.0
236,HOUSEHOLD_1_009_CA_1_validation,0.3,NaN,NaN,NaN,NaN,0.0,0.0
237,HOUSEHOLD_1_009_CA_1_validation,0.4,NaN,NaN,NaN,NaN,0.0,0.0
238,HOUSEHOLD_1_009_CA_1_validation,0.5,NaN,NaN,NaN,NaN,0.0,0.0
...,...,...,...,...,...,...,...,...
31369,HOUSEHOLD_2_515_CA_4_validation,0.5,NaN,NaN,NaN,NaN,0.0,0.0
31370,HOUSEHOLD_2_515_CA_4_validation,0.6,NaN,NaN,NaN,NaN,0.0,0.0
31371,HOUSEHOLD_2_515_CA_4_validation,0.7,NaN,NaN,NaN,NaN,0.0,0.0
31372,HOUSEHOLD_2_515_CA_4_validation,0.8,NaN,NaN,NaN,NaN,0.0,0.0


In [22]:
df_result_metrics_all_params[df_result_metrics_all_params['MAPE'].isin([np.inf, -np.inf, np.nan])]

,Product,Alpha,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE
9,HOUSEHOLD_1_001_CA_2_validation,0.1,inf,inf,200.0,NaN,0.069348,0.069348
10,HOUSEHOLD_1_001_CA_2_validation,0.2,inf,inf,200.0,NaN,0.068120,0.068120
11,HOUSEHOLD_1_001_CA_2_validation,0.3,inf,inf,200.0,NaN,0.067659,0.067659
12,HOUSEHOLD_1_001_CA_2_validation,0.4,inf,inf,200.0,NaN,0.067935,0.067935
13,HOUSEHOLD_1_001_CA_2_validation,0.5,inf,inf,200.0,NaN,0.068966,0.068966
...,...,...,...,...,...,...,...,...
31369,HOUSEHOLD_2_515_CA_4_validation,0.5,NaN,NaN,NaN,NaN,0.000000,0.000000
31370,HOUSEHOLD_2_515_CA_4_validation,0.6,NaN,NaN,NaN,NaN,0.000000,0.000000
31371,HOUSEHOLD_2_515_CA_4_validation,0.7,NaN,NaN,NaN,NaN,0.000000,0.000000
31372,HOUSEHOLD_2_515_CA_4_validation,0.8,NaN,NaN,NaN,NaN,0.000000,0.000000


In [23]:
df_result_metrics_all_params[df_result_metrics_all_params['RMSE'].isin([np.inf, -np.inf, np.nan])]

,Product,Alpha,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE


In [24]:
df_result_metrics_all_params[df_result_metrics_all_params['MAE'].isin([np.inf, -np.inf, np.nan])]

,Product,Alpha,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE


# Metrics Statistics

In [25]:
list_metrics = ['MASE', 'WMAPE', 'SMAPE', 'MAPE', 'MAE', 'RMSE']

In [26]:
for mts in list_metrics:
    print('Percentage of unexpected values of', mts, 'is: {}'.format(df_result_metrics_all_params[df_result_metrics_all_params[mts].isin([np.nan, np.inf, -np.inf])].Product.nunique() / df_result_metrics_all_params.Product.nunique() * 100), "%")

Percentage of unexpected values of MASE is: 23.667621776504298 %
Percentage of unexpected values of WMAPE is: 23.667621776504298 %
Percentage of unexpected values of SMAPE is: 22.836676217765042 %
Percentage of unexpected values of MAPE is: 23.667621776504298 %
Percentage of unexpected values of MAE is: 0.0 %
Percentage of unexpected values of RMSE is: 0.0 %


### Filter all rows that have unexpected metrics values

In [27]:
df_result_metrics_all_params = df_result_metrics_all_params[~df_result_metrics_all_params.isin([np.nan, np.inf, -np.inf]).any(1)]

### Get MEAN metrics value of each Alpha

In [28]:
df_result_metrics_all_params.groupby(['Alpha'])['MASE', 'WMAPE', 'SMAPE', 'MAPE', 'MAE', 'RMSE'].mean()

,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE
Alpha,,,,,,
0.1,1.180717,176.806849,154.461492,61.902982,0.707148,0.900766
0.2,1.158160,171.144560,154.569714,62.766279,0.714284,0.909786
0.3,1.165571,170.838094,154.732619,63.286247,0.728542,0.926234
0.4,1.184770,172.694046,154.916751,63.749036,0.745798,0.946318
0.5,1.211841,175.977876,155.096467,64.201127,0.765045,0.969612
0.6,1.246706,180.600380,155.272811,64.680094,0.786465,0.996521
0.7,1.291548,186.860452,155.434291,65.220518,0.810754,1.028076
0.8,1.351404,195.511211,155.558048,65.794756,0.838795,1.066392
0.9,1.439150,208.402107,155.585634,66.371795,0.873189,1.116648


In [29]:
for mts in list_metrics:
    print('The optimum Alpha based on Mean', mts, 'is: {}'.format(df_result_metrics_all_params.groupby(['Alpha'])['MASE', 'WMAPE', 'SMAPE', 'MAPE', 'MAE', 'RMSE'].mean()[mts].idxmin()))

The optimum Alpha based on Mean MASE is: 0.2
The optimum Alpha based on Mean WMAPE is: 0.3
The optimum Alpha based on Mean SMAPE is: 0.1
The optimum Alpha based on Mean MAPE is: 0.1
The optimum Alpha based on Mean MAE is: 0.1
The optimum Alpha based on Mean RMSE is: 0.1


### Get MEDIAN metrics value of each Alpha

In [30]:
df_result_metrics_all_params.groupby(['Alpha'])['MASE', 'WMAPE', 'SMAPE', 'MAPE', 'MAE', 'RMSE'].median()

,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE
Alpha,,,,,,
0.1,0.930586,133.8587,165.1095,61.6139,0.628172,0.771092
0.2,0.939881,136.9100,164.6036,62.5064,0.632127,0.780842
0.3,0.952211,139.3003,164.2264,62.5047,0.645380,0.793180
0.4,0.972560,142.4974,163.5010,62.2382,0.655562,0.809515
0.5,0.992171,145.8073,162.9854,62.2283,0.668791,0.826317
0.6,1.015876,149.1944,162.7181,62.4603,0.682812,0.845006
0.7,1.040911,153.4695,162.4234,62.8012,0.702047,0.868602
0.8,1.068972,157.7113,162.2341,63.4447,0.728365,0.898181
0.9,1.102768,162.6259,161.8984,64.1879,0.752780,0.940348


In [31]:
for mts in list_metrics:
    print('The optimum Alpha based on Median of', mts, 'is: {}'.format(df_result_metrics_all_params.groupby(['Alpha'])['MASE', 'WMAPE', 'SMAPE', 'MAPE', 'MAE', 'RMSE'].median()[mts].idxmin()))

The optimum Alpha based on Median of MASE is: 0.1
The optimum Alpha based on Median of WMAPE is: 0.1
The optimum Alpha based on Median of SMAPE is: 0.9
The optimum Alpha based on Median of MAPE is: 0.1
The optimum Alpha based on Median of MAE is: 0.1
The optimum Alpha based on Median of RMSE is: 0.1


# Run Croston model after decide best paramaters

In [32]:
best_alpha = 0.1

In [33]:
def Croston_test_data(best_alpha, sales_pattern, start_train_date, end_train_date, 
                       n_pred_days, num_train_needed, validation_training_ratio):
    
    sales_pattern_py = sales_pattern.copy()
    list_pattern_py = sales_pattern_py.id.unique().tolist()
    
    sales_pattern_py = sales_pattern_py.drop(['item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'], axis=1)
    df_pattern = sales_pattern_py.melt(['id'], var_name='Date').pivot(index = ['Date'], columns = 'id', values = 'value').reset_index()
    df_pattern.drop('Date', axis=1, inplace=True)
    df_pattern.index = pd.date_range('2011-01-29', periods=df_pattern.shape[0], freq="D")
    df_pattern.index.names = ['Date']
    df_pattern = df_pattern.astype('float64')
    
    train_data = df_pattern[(df_pattern.index >= start_train_date) & (df_pattern.index <= end_train_date)]
    test_data = df_pattern.iloc[n_pred_days*-1:]
    
    df_result_best_params = pd.DataFrame()
    for product in list_pattern_py:
        train_product = train_data[product]
        test_product = test_data[product]
        predictions = list()
        history = [x for x in train_product[num_train_needed*-1:]]
        for t in range(len(test_product)):
            yhat = Croston(history, extra_periods=1, alpha=best_alpha)['Forecast'].iloc[-1]
            predictions.append(yhat)
            history.append(test_product[t])
        df_result_temp = pd.DataFrame({'Product': [product for count in range(len(test_product))],
                                        'Actual Data': test_product,
                                        'Forecast': predictions})
        df_result_best_params = df_result_best_params.append(df_result_temp, ignore_index=True)
            
    return df_result_best_params

In [34]:
start = timer()

pattern_df_result_best_params = Croston_test_data(best_alpha, sales_intermittent, start_train_date, end_train_date, n_pred_days, num_train_needed, validation_training_ratio)

end = timer()

print('This line of code took {} minutes'.format((end-start) / 60))

This line of code took 0.47037211403333384 minutes


In [35]:
df_result_final = pattern_df_result_best_params.groupby('Product').apply(get_metrics_result_all_params).reset_index()

In [36]:
df_result_final

,Product,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE
0,HOUSEHOLD_1_001_CA_1_validation,1.059130,152.0803,187.0705,82.2310,0.325886,0.416469
1,HOUSEHOLD_1_001_CA_2_validation,0.903625,194.6269,197.4026,90.0000,0.139019,0.252939
2,HOUSEHOLD_1_001_CA_3_validation,0.689993,115.5886,139.2941,42.1479,0.743070,0.901193
3,HOUSEHOLD_1_001_CA_4_validation,inf,inf,200.0000,NaN,0.050403,0.050403
4,HOUSEHOLD_1_002_CA_1_validation,0.868989,140.3751,195.9870,92.4468,0.200536,0.357159
...,...,...,...,...,...,...,...
3485,HOUSEHOLD_2_515_CA_4_validation,NaN,NaN,NaN,NaN,0.000000,0.000000
3486,HOUSEHOLD_2_516_CA_1_validation,1.336559,143.9372,195.7549,91.9748,0.205625,0.356762
3487,HOUSEHOLD_2_516_CA_2_validation,inf,inf,200.0000,NaN,0.383959,0.383959
3488,HOUSEHOLD_2_516_CA_3_validation,1.781358,191.8386,198.0851,92.8163,0.137028,0.258694


In [37]:
df_result_final.isnull().sum()

Product       0
MASE        737
WMAPE       737
SMAPE       788
MAPE       1058
MAE           0
RMSE          0
dtype: int64

In [38]:
df_result_final.to_csv('Croston_Intermittent_Test_Data.csv')